In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def func(x):
    return 2 - 2 * x
 

In [ ]:
def draw_samples_exact(u):
    x = 1 - np.sqrt(1 - u)
    pdf = 2 - 2 * x
    return x, pdf

def draw_samples_p1(u):
    return u, np.ones_like(u)

def draw_samples_p2(u):
    lower_bound = 1e-4
    lower_bound = 0
    norm = 1 + lower_bound*0.5
    x = np.where(u<=1/norm, u*norm/2, (u*norm-1)/lower_bound + 0.5)
    pdf = np.where(x<=0.5, 2, lower_bound) / norm
    return x, pdf

In [ ]:
num_samples = 1024*64
u = np.linspace(0, 1, num_samples)
x_p1, pdf_p1 = draw_samples_p1(u)
x_p2, pdf_p2 = draw_samples_p2(u)
x_exact, pdf_exact = draw_samples_exact(u)

plt.figure()
plt.plot(u, x_p1, label="p1")
plt.plot(u, x_p2, label="p2")
plt.plot(u, x_exact, label="exact")
plt.legend()


plt.figure()
plt.xlim(0, 1)
plt.ylim(0, 2.5)
plt.plot(u, 1/pdf_p1, label="p1")
plt.plot(u, 1/pdf_p2, label="p2")
plt.plot(u, 1/pdf_exact, label="exact")
plt.legend()

plt.figure()
plt.plot(x_p1, pdf_p1, label="p1")
plt.plot(x_p2, pdf_p2, label="p2")
plt.plot(x_exact, pdf_exact, label="exact")
plt.legend()



In [ ]:
def integrate_func(sample_func, num_samples):
    u = np.linspace(0, 1, num_samples)
    x, pdf = sample_func(u)
    y = func(x)
    return np.sum(np.where(pdf > 0, y / pdf, 0)) / num_samples

print(integrate_func(draw_samples_p1, 4096 * 128))
print(integrate_func(draw_samples_p2, 4096 * 128))
print(integrate_func(draw_samples_exact, 4096 * 128))


In [ ]:
num_samples = 1024*8
delta = 1/num_samples
u = np.arange(0, 1, delta) + np.random.random_sample([int(num_samples)])*delta
x_p1, pdf_p1 = draw_samples_p1(u)
x_p2, pdf_p2 = draw_samples_p2(u)
x_exact, pdf_exact = draw_samples_exact(u)

plt.figure()
plt.xlim(0, 1)
plt.ylim(0, 3.5)
plt.plot(x_p1, 1/pdf_p1, label="p1")
plt.plot(x_p2, 1/pdf_p2, label="p2")
plt.plot(x_exact, 1/pdf_exact, label="exact")
plt.legend()


In [ ]:
num_samples = 1024
M = 32*32
num_rands = (int)(num_samples * M / 2)

x_p1, pdf_p1 = draw_samples_p1(np.random.random_sample(num_rands))
x_p2, pdf_p2 = draw_samples_p2(np.random.random_sample(num_rands))
x = np.append(x_p1, x_p2)
pdf = np.append(pdf_p1, pdf_p2)

shuffle_idx = np.arange(num_samples*M).astype(int)
np.random.shuffle(shuffle_idx)
x = x[shuffle_idx]
pdf = pdf[shuffle_idx]

target_pdf = func(x)
weight = np.where(pdf>0, target_pdf/pdf, 0)


reservoir = np.zeros(num_samples)
weight_sum = np.zeros(num_samples)
Z = np.zeros(num_samples)

for i in range(M):
    idx = np.arange(0, num_samples) + i * num_samples

    xi = x[idx]
    w = weight[idx]
    weight_sum += weight[idx]

    Z = Z + np.where(pdf[idx]>0, 1, 0)
    u = np.random.random_sample(num_samples)
    reservoir = np.where(u * weight_sum < w, xi, reservoir)

m = 1/M
#m = 1/Z
W = 1.0/func(reservoir) * (m * weight_sum)

sort_indices = np.argsort(reservoir)
plt.plot(reservoir[sort_indices], W[sort_indices], label="foo")

plt.plot(x_exact, 1/pdf_exact, label="reference")
plt.ylim(0, 3.2)
plt.legend()
